In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from dataset import DatasetConfig, DatasetBuilder
import os
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import defaultdict
from typing import Callable, Dict, List, Tuple
%matplotlib inline

In [6]:
ds_config =  DatasetConfig(model_dir='../dataset/pinecone/sparse/0/',
                           images_dir='../dataset/pinecone/images/',
                           batch_from_single_image=False,
                           is_training=True,
                           batch_size=64)

In [7]:
dataset = DatasetBuilder(ds_config)
ds = dataset.build()

In [8]:
lines = []
colors = []

def make_line(origin, direction, length=10):
    return origin, origin + direction * length

for elements in ds.take(100).as_numpy_iterator():
    origins = elements["origins"]
    directions = elements["directions"]
    pixels = elements["pixels"]
    N = origins.shape[0]
    for i in range(N):
        p0, p1 = make_line(origins[i, ...], directions[i, ...])
        lines.append(p0)
        lines.append(p1)
        colors.append(pixels[i, ...])
    print("Done")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [9]:
data = np.asarray(lines)
np.save('lines.npy', data)

colors = np.asarray(colors)
np.save('colors.npy', colors)

In [52]:
t = {k:str(k) for k in range(10)}

def convert_table(table):
    init = tf.lookup.KeyValueTensorInitializer(list(table.keys()), list(table.values()))
    tf_table = tf.lookup.StaticHashTable(
        init,
        default_value="")
    return tf_table

ttf = convert_table(t)

@tf.function
def map_fn(i):
    return ttf[i]

def numpy_fn(x):
    

In [12]:
class Foo:
    def __init__(self, x):
        self.x = x

t = {k:Foo(k) for k in range(10)}

def map_fn(i):
    x = tf.convert_to_tensor([t[i].x], dtype=tf.float32)
    x2 = x * 2
    return x, x2

def py_fn(x):
    x, x2 = tf.py_function(map_fn, [x], [tf.float32, tf.float32])
    return {"x":x, "x2":x2}

In [52]:
map_fn(0)

(0, 0)

In [13]:
image_indices = list(range(10))
ds = tf.data.Dataset.from_tensor_slices(image_indices)

ds = ds.map(map_fn)

TypeError: in user code:

    <ipython-input-12-1c2d725436db>:8 map_fn  *
        x = tf.convert_to_tensor([t[i].x], dtype=tf.float32)
    /home/liangkai/.local/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:845 __hash__
        raise TypeError("Tensor is unhashable. "

    TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.


In [8]:
for i in ds.take(10):
    print(i)

{'x': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, 'x2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>}
{'x': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, 'x2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>}
{'x': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>, 'x2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>}
{'x': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.], dtype=float32)>, 'x2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([6.], dtype=float32)>}
{'x': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>, 'x2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.], dtype=float32)>}
{'x': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>, 'x2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([10.], dtype=float32)>}
{'x': <tf.Tensor: shape=(1,), dtype=flo

In [53]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [21]:
b

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 7, 8, 4, 5, 3, 2, 9, 6], dtype=int32)>